# Importing Modules and Defining Functions

In [1]:
import re
import chess
import chess.pgn
import sys
import requests
import pandas as pd
from json import loads
import matplotlib.pyplot as plt

In [2]:
def httpsReq(requestAddr):

    #Making the API Request
    # print(f'Making API Call to: {requestAddr}')
    response = requests.get(requestAddr)
    if response.status_code == 200:
        pass
    elif response.status_code != 200:
        print(f'{response.status_code} - API Request {requestAddr} Failed')
        KeyboardInterrupt

    #Converting JSON to Dict and Parsing API Data into a Dataframe
    api_dict = loads(response.text)

    return api_dict

In [3]:
#Function for Writing .xlsx Data Tables which will provide a standard output for table formatting 
def xlsx_table_writer(data_table,sheet_name,col_width_list,title_str,startrow,startcol,worksheet,workbook,writer):

    #Layout/Formatting
    t_vert_spacing = 5 #Vertical Spacing between tables
    t_horz_spacing = 1 #Horizontal Spacing between tables
    title_format = workbook.add_format({'bold': True, 'font_size' : 20, 'fg_color' : '#76933C', 'font_color' : 'white' }) #Standardised Title Format for all tables
    header_format = workbook.add_format({'bold' : True, 'font_size' : 12, 'text_wrap' : True}) #text-wrapping for table headers
    
    # Adding and changing active sheet
    try:
        worksheet=workbook.add_worksheet(sheet_name)
        writer.sheets[sheet_name] = worksheet 
        worksheet.set_zoom(70)
        startrow = 3
        startcol = 1
    except:
        pass

    for i in range(len(col_width_list)):
        worksheet.set_column(i+startcol, i+startcol, col_width_list[i])

    #Writing in Cell Data and Merging Cells for Table Titles
    data_table.to_excel(writer,sheet_name=sheet_name,startrow=startrow , startcol=startcol, index=False, header=False)
    worksheet.merge_range(startrow-2, startcol,startrow-2,data_table.shape[1] + startcol -1, title_str,title_format) #writing in title formatting above table

    #Column settings to use in add table function
    column_settings = [{'header' : column} for column in data_table.columns]

    #Populating Excel with Table Format - Adding table to xls for each df
    worksheet.add_table(startrow-1, startcol, startrow + data_table.shape[0], data_table.shape[1] + startcol - 1, {'columns' : column_settings, 'style': 'Table Style Medium 4', 'autofilter' : False})   

    #Applying a text wrap to the Column Header
    for col_num, value in enumerate(data_table.columns.values):
        worksheet.write(startrow-1, col_num + startcol, value, header_format)
    
    #Setting Positions of Following Tables Insertions
    startrow = startrow + data_table.shape[0] + t_vert_spacing #Setting to start row for next table
    # startrow = 3  
    # startcol = startcol + data_table.shape[1] + t_horz_spacing  #Disabling horizontally displaced tables in favour of vertically displacements
    startcol = 1

    #Setting the column width at the end of the table to keep to spacing minimal between tables
    worksheet.set_column(startcol-1,startcol-1,1)
    
    #Return the start row in order to index for future function calls
    return [startrow,startcol,worksheet,workbook,writer]

In [4]:
#Function to Detect Operating System and Adjust Pathing to Respective Filesystem
def pathing(folder_path,filename):

    #Windows Operating System
    if 'win' in sys.platform:
        if folder_path == 'root':
            filepath = f'{sys.path[0]}\\{filename}'
        else:
            filepath = f'{sys.path[0]}\\{folder_path}\\{filename}'
    #Linux/Mac Operating Sytem
    else:
        if folder_path == 'root':
            filepath = f'{sys.path[0]}/{filename}'
        else:
            filepath = f'{sys.path[0]}/{folder_path}/{filename}'

    return filepath

# Requesting Game History via Chess.com API
### We need to stage our api request to transform into a dataframe:
- The api request address only returns the games for the selected month and year
- There we must pull down our archive histotry and loop through the api requests to return data for each month

In [5]:
#Retrieveing Archive of My Entire Chess.com History
player = 'filiplivancic'
dict = httpsReq(f'https://api.chess.com/pub/player/{player}/games/archives')
print(f'API Request for Monthly API Endpoints Successful')
archive = (dict['archives'])
archive[:5]

API Request for Monthly API Endpoints Successful


['https://api.chess.com/pub/player/filiplivancic/games/2019/05',
 'https://api.chess.com/pub/player/filiplivancic/games/2019/07',
 'https://api.chess.com/pub/player/filiplivancic/games/2019/08',
 'https://api.chess.com/pub/player/filiplivancic/games/2019/09',
 'https://api.chess.com/pub/player/filiplivancic/games/2019/10']

In [6]:
api_list = [httpsReq(request) for request in archive]
print(f'API Request Successful and Appended all request links together')

API Request Successful and Appended all request links together


## Unpacking List of Games

In [7]:
game_list = []
print('Transforming JSON data into a dataframe')
for month in api_list: 
    month_of_games = month['games']
    for game in month_of_games: 
        game_list.append(game)
df = pd.DataFrame(game_list)

Transforming JSON data into a dataframe


### Filtering the rule set strictly to 'chess'
- There are other subsets of rules such as **'crazyhouse'** which add columns to the pgn and alter the shape of the table - *we just want the defualt chess pgn's*
- Crazyhouse rules add cols 23,24,25 which relate to the "Varient URL", "unknown blank col", "varient chess moves"
- df.loc[df['rules']=='crazyhouse']

In [8]:
df = df.loc[df['rules']=='chess']
df.reset_index(drop=True,inplace=True)
df.head(3)

,url,pgn,time_control,end_time,rated,tcn,uuid,initial_setup,fen,time_class,rules,white,black,accuracies
0,https://www.chess.com/game/live/3668045802,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556902183,True,mC3Nbs5Qlt!TcMZJCKQKtB7RfH6Zdm86BKRzKT0THOzjOX...,fd899364-2d30-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,7r/pkp2pp1/8/3p3p/4r3/8/P1P1KPPP/q3b3 w - -,blitz,chess,"{'rating': 804, 'result': 'timeout', '@id': 'h...","{'rating': 973, 'result': 'win', '@id': 'https...",NaN
1,https://www.chess.com/game/live/3668084540,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556902975,True,lB!Tbs5QBJQKnDKEpxEVmC0SCKT!sC9zclWGkA7FowF0iq...,a090dbf8-2d36-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,2k1r2r/1pp4p/3p1p1n/P2P1n2/4K1B1/6q1/1P1B4/R2Q...,blitz,chess,"{'rating': 797, 'result': 'checkmated', '@id':...","{'rating': 925, 'result': 'win', '@id': 'https...","{'white': 22.005480522744918, 'black': 56.4909..."
2,https://www.chess.com/game/live/3668146117,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1556903918,True,ltWGcM2Ugv4OmCYItBIAfAOPbsPQBJQAvKAYdvZRv1,96bfa632-2d3f-11e5-8000-000000010001,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,1nbqkbnr/1pr1pQ1p/3p2p1/p2PN1B1/4P3/2N5/PPP2PP...,blitz,chess,"{'rating': 990, 'result': 'win', '@id': 'https...","{'rating': 775, 'result': 'checkmated', '@id':...",NaN


#### Creating a subset of data with select headers (Used later to merge in with pgn data)

In [107]:
df_subset_attributes = df.loc[:,['url','rated','rules','time_class']]

# Re-naming the url column to enable a merging criterion down the script for the pgn table
if 'url' in df_subset_attributes.columns:
    df_subset_attributes.rename(columns={'url':'Link'},inplace=True)

df_subset_attributes[:3]

,Link,rated,rules,time_class
0,https://www.chess.com/game/live/3668045802,True,chess,blitz
1,https://www.chess.com/game/live/3668084540,True,chess,blitz
2,https://www.chess.com/game/live/3668146117,True,chess,blitz


#### Cleaning pgn data in preparation for putting into dataframe
- Splitting new lines into lists
- Brackets and slashed removed

In [78]:
pgn_list = [df.loc[i,'pgn'].splitlines(False) for i in range(df.shape[0])] #splitting newlines into list structure

for i,pgn in enumerate(pgn_list):

    for j in range(len(pgn)):
        pgn_list[i][j] = pgn_list[i][j].split(' \"')

        for k in range(len(pgn_list[i][j])):
            pgn_list[i][j][k] = pgn_list[i][j][k].replace('\"','')
            pgn_list[i][j][k] = pgn_list[i][j][k].replace('[','')
            pgn_list[i][j][k] = pgn_list[i][j][k].replace(']','')

#### Retrieving pgn headers

In [79]:
for pgn in pgn_list[0]:
    headers = [pgn[0] if len(pgn[0]) < 20 else 'Moves' for pgn in pgn_list[0]]

print(len(headers),headers)

23 ['Event', 'Site', 'Date', 'Round', 'White', 'Black', 'Result', 'CurrentPosition', 'Timezone', 'ECO', 'ECOUrl', 'UTCDate', 'UTCTime', 'WhiteElo', 'BlackElo', 'TimeControl', 'Termination', 'StartTime', 'EndDate', 'EndTime', 'Link', '', 'Moves']


#### Creating DataFrame and replacing list pairs with just the values

In [80]:
df_pgn = pd.DataFrame(pgn_list,columns=headers)
for col_head in headers:
    # print(f'Replacing List Pairs with Values For Column: {col_head}')
    for i,list_pair in enumerate(df_pgn[col_head]):
        if len(list_pair) == 2:
            df_pgn.at[i,col_head]  = list_pair[1]
        elif len(list_pair) == 1:
            df_pgn.at[i,col_head]  = list_pair[0]
df_pgn.head(3)

,Event,Site,Date,Round,White,Black,Result,CurrentPosition,Timezone,ECO,...,WhiteElo,BlackElo,TimeControl,Termination,StartTime,EndDate,EndTime,Link,,Moves
0,Live Chess,Chess.com,2019.05.03,-,filiplivancic,Jacoblb56,0-1,7r/pkp2pp1/8/3p3p/4r3/8/P1P1KPPP/q3b3 w - -,UTC,B00,...,804,973,600,Jacoblb56 won on time,16:34:35,2019.05.03,16:49:43,https://www.chess.com/game/live/3668045802,,1. e4 {%clk 0:09:55.5} 1... h5 {%clk 0:09:57.9...
1,Live Chess,Chess.com,2019.05.03,-,Mazterja5,filiplivancic,0-1,2k1r2r/1pp4p/3p1p1n/P2P1n2/4K1B1/6q1/1P1B4/R2Q...,UTC,A45,...,797,925,600,filiplivancic won by checkmate,16:49:51,2019.05.03,17:02:55,https://www.chess.com/game/live/3668084540,,1. d4 {%clk 0:09:59.7} 1... Nf6 {%clk 0:09:56....
2,Live Chess,Chess.com,2019.05.03,-,filiplivancic,jakubreid,1-0,1nbqkbnr/1pr1pQ1p/3p2p1/p2PN1B1/4P3/2N5/PPP2PP...,UTC,A00,...,990,775,600,filiplivancic won by checkmate,17:13:56,2019.05.03,17:18:38,https://www.chess.com/game/live/3668146117,,1. d3 {%clk 0:09:52.8} 1... a5 {%clk 0:09:57.2...


#### Replace Opening URL's with Chess Opening Names

In [81]:
df_pgn['ECOUrl'] = df_pgn['ECOUrl'].str.replace('https://www.chess.com/openings/','',regex=True)
df_pgn['ECOUrl'] = df_pgn['ECOUrl'].str.replace('-',' ',regex=True)
df_pgn['ECOUrl']

0                    Kings Pawn Opening Goldsmith Defense
1                                       Indian Game 2.Nc3
2                                          Mieses Opening
3                          Modern Defense with 1 e4 2.Bc4
4                                       Caro Kann Defense
                              ...                        
2516           French Defense Exchange Variation 3...Qxd5
2517            French Defense Advance Nimzowitsch System
2518    Nimzowitsch Defense Declined Franco Nimzowitsc...
2519                               Philidor Defense 3.Bc4
2520           French Defense Exchange Variation 3...Qxd5
Name: ECOUrl, Length: 2521, dtype: object

#### Transform Date Columns into Datetime datatype

In [109]:
# # Converting to Datetime Format
df_pgn['Date'] = pd.to_datetime(df_pgn['Date'])
df_pgn['EndDate'] = pd.to_datetime(df_pgn['EndDate'])

#### Merging chess pgn dataframe back into the main dataframe via the game url/link

In [110]:
df_original = pd.merge(df_pgn,df_subset_attributes,on='Link',how='inner')
print(f'There are {df_original.shape[1]} cols:\n{list(df_original.columns)}')
df_original.shape[0]

There are 26 cols:
['Event', 'Site', 'Date', 'Round', 'White', 'Black', 'Result', 'CurrentPosition', 'Timezone', 'ECO', 'ECOUrl', 'UTCDate', 'UTCTime', 'WhiteElo', 'BlackElo', 'TimeControl', 'Termination', 'StartTime', 'EndDate', 'EndTime', 'Link', '', 'Moves', 'rated', 'rules', 'time_class']


2521

In [111]:
set(df_original['TimeControl'])

{'180', '180+2', '1800', '300', '60', '600', '900+10'}

#### Filtering Ranked Games

In [112]:
# Filtering Down to Rated Games to specific time classifications
df_blitz_600 = df_original.loc[
                                    (df_original['time_class']=='blitz')
                                  & (df_original['TimeControl']=='600')
                                  & (df_original['rated'] == True)
                                  ]

df_blitz_180_2 = df_original.loc[
                                    (df_original['time_class']=='blitz')
                                  & (df_original['TimeControl']=='180+2')
                                  & (df_original['rated'] == True)
                                  ]
df_blitz_600[:3]

,Event,Site,Date,Round,White,Black,Result,CurrentPosition,Timezone,ECO,...,Termination,StartTime,EndDate,EndTime,Link,,Moves,rated,rules,time_class
0,Live Chess,Chess.com,2019-05-03,-,filiplivancic,Jacoblb56,0-1,7r/pkp2pp1/8/3p3p/4r3/8/P1P1KPPP/q3b3 w - -,UTC,B00,...,Jacoblb56 won on time,16:34:35,2019-05-03,16:49:43,https://www.chess.com/game/live/3668045802,,1. e4 {%clk 0:09:55.5} 1... h5 {%clk 0:09:57.9...,True,chess,blitz
1,Live Chess,Chess.com,2019-05-03,-,Mazterja5,filiplivancic,0-1,2k1r2r/1pp4p/3p1p1n/P2P1n2/4K1B1/6q1/1P1B4/R2Q...,UTC,A45,...,filiplivancic won by checkmate,16:49:51,2019-05-03,17:02:55,https://www.chess.com/game/live/3668084540,,1. d4 {%clk 0:09:59.7} 1... Nf6 {%clk 0:09:56....,True,chess,blitz
2,Live Chess,Chess.com,2019-05-03,-,filiplivancic,jakubreid,1-0,1nbqkbnr/1pr1pQ1p/3p2p1/p2PN1B1/4P3/2N5/PPP2PP...,UTC,A00,...,filiplivancic won by checkmate,17:13:56,2019-05-03,17:18:38,https://www.chess.com/game/live/3668146117,,1. d3 {%clk 0:09:52.8} 1... a5 {%clk 0:09:57.2...,True,chess,blitz


In [113]:

# #Applying to filtering to results_df and reseting index to make games appear sequentially
# result_df = result_df[result_df.index.isin(df_blitz_rating.index)].reset_index()
# result_df

## Applying Data Aggregation Techniques

In [114]:
# #Grouping Most Played Chess Openings
# df_openings = df_blitz_rating.groupby(['Opening'])['rated'].count().reset_index(name='Games')
# df_openings = df_openings.sort_values(by=['Games'],ascending=False)
# df_openings.head(10)

In [115]:
# # Grouping together Wins and Losses (For BLitz)
# def percentage_string(value):
#     return "{:.2%}".format(value)

# df_winLoss = df_blitz_rating.groupby(['win/loss'])['rated'].count().reset_index(name='Games')
# win_rate = percentage_string(df_winLoss.at[2,'Games']/df_blitz_rating.shape[0])
# loss_rate = percentage_string(df_winLoss.at[1,'Games']/df_blitz_rating.shape[0])
# draw_rate = percentage_string(df_winLoss.at[0,'Games']/df_blitz_rating.shape[0])

# #Appending Win Loss Rates onto DataFrame
# rates_list = [draw_rate,loss_rate,win_rate]
# df_winLoss['Ratio'] = rates_list

# print(f'---{player} Chess Stats--- \n{df_blitz_rating.shape[0]} Ranked Blitz Games Played:\n  Win Rate: {(win_rate)}\n  Lose Rate: {loss_rate}\n  Draw Rate: {draw_rate}\n-------------------------------')
# # df_winLoss


# Plotting the Rating Change Over Time



In [116]:
# fig, ax = plt.subplots(figsize=(10,6))
# ax.plot(result_df.index,result_df['rating'])
# ax.set_title(f'{player} Chess Rating History - Time Control: 10 Minutes',loc='left',fontsize=16)
# ax.set_xlabel('Number of Rapid Games Played', fontsize = 10)
# ax.set_ylabel('Chess Rating', fontsize=10)
# plt.tight_layout()

# #Saving Chart
# plt.savefig(pathing('pictures','rating_hist.png'),dpi=100)


## Applying a moving Average on the Rating Data

In [117]:
# #Applying 50 point moving average
# result_avg_df = result_df.copy()
# window_size = 50
# avg_data= []
# for i in range(result_avg_df.shape[0]):
#     mov_avg = result_avg_df.loc[i:(i+window_size),'rating'].mean()
#     avg_data.append(mov_avg)

# #Appending list to dataframe
# result_avg_df['average_rating'] = avg_data

# #Plotting Averaged Data
# fig, ax = plt.subplots(figsize=(10,6))
# ax.plot(result_avg_df.index,result_avg_df['average_rating'])
# ax.set_title(f'{player} Chess Rating History -  50 Point Moving Average - Time Control: 10 Minutes',loc='left',fontsize=14)
# ax.set_xlabel('Number of Rapid Games Played', fontsize = 10)
# ax.set_ylabel('Chess Rating', fontsize=10)
# plt.tight_layout()

# #Saving Chart
# plt.savefig(pathing('pictures','rating_hist_avg.png'),dpi=100)

## Initialising XlsxWriter Workbook And Exporting Tables to .xlsx

In [118]:
# #----------------------------------------------------------------------------------------------------------------------
# #--------------------------------------------.xlsx workbook initialisation---------------------------------------------
# #----------------------------------------------------------------------------------------------------------------------

# #Excel Output Filepath
# excelpath = pathing('xlsx_data','chess_data.xlsx')

# #Generating .xlsx workbook in which to record dataframes for usage.
# writer = pd.ExcelWriter(excelpath,engine='xlsxwriter')   
# workbook=writer.book
# worksheet='' #Initialise variable
# startrow = 0 #Initialise variable
# startcol = 0 #Initialise variable
# v_idx = 0    #initialise counter to index vertical chart insertions
# lr_pos = 0   #initialise left right insertion variable

# #----------------------------------------------------------------------------------------------------------------------
# #----------------------------------------------------------------------------------------------------------------------

In [119]:
# #Summary Table
# data_table = df_winLoss
# sheet_name = 'Summary'
# title_str = f'Chess Summary Stats - {player}'
# col_width_list = [20,20,20]
# [startrow,startcol,worksheet,workbook,writer] = xlsx_table_writer(data_table,sheet_name,col_width_list,title_str,startrow,startcol,worksheet,workbook,writer)

In [120]:
# #Openings Table
# data_table = df_openings
# sheet_name = 'Openings'
# title_str = f'Chess Opening Count - {player}'
# col_width_list = [40,20,20]
# [startrow,startcol,worksheet,workbook,writer] = xlsx_table_writer(data_table,sheet_name,col_width_list,title_str,startrow,startcol,worksheet,workbook,writer)

In [121]:
# #Results Table
# data_table = result_avg_df
# sheet_name = 'Rating History'
# title_str = f'Chess Rating History - {player}'
# col_width_list = [7,7,17,9,22,15]
# [startrow,startcol,worksheet,workbook,writer] = xlsx_table_writer(data_table,sheet_name,col_width_list,title_str,startrow,startcol,worksheet,workbook,writer)

In [122]:
# #Save into Excel and Exit Python out of File
# writer.close()